In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

In [2]:
dataset_names = [
         'adult',
         'Amazon_employee_access',
         'bank-marketing',
         'Click_prediction_small',
         'credit-g',
         ]

In [3]:
auto_ml_names = [
    'Auto_ml', 
    'h2o', 
    'tpot', 
    'catboost-default',
    'lightgbm-default',
    'lightgbm-opt',
    ]

In [4]:
METRIC = 'AUC'
metrics = ['AUC', 'log_loss', 'Accuracy']

In [5]:
bench_result = {}
for name in auto_ml_names:
    bench_result[name] = 0

In [15]:
total_place = pd.DataFrame()
total_score = pd.DataFrame()
#for metric in metrics:
metric = 'AUC'
bench_result = {}

for name in auto_ml_names:
    bench_result[name] = 0
for dataset_name in dataset_names:
    print(dataset_name)
    bench_auc = []
    for AUTOML in auto_ml_names:
        result = pd.read_csv(f'./frameworks/{AUTOML}/result/{dataset_name}_{AUTOML}_metrics.csv')
        bench_auc.append(result[metric].mean())
    res = pd.DataFrame(bench_auc, columns=[metric,], index=auto_ml_names).sort_values(by=[metric])
    for i, name in enumerate(res.index):
        bench_result[res.index[i]] += i+1
#total_place.append(bench_result)
        bench = pd.io.json.json_normalize(bench_result,).T
        bench.columns = [f'{metric} place']
    res.columns=[dataset_name,]
    total_score = pd.concat([total_score, res], axis=1, sort=False)

    
total_place = pd.concat([total_place, bench], axis=1, sort=False)
        #bench = bench.sort_values(by=f'{metric} place', ascending=False)
        #bench.to_csv(f'{metric}_place.csv',)
#result.head(5)

adult
Amazon_employee_access
bank-marketing
Click_prediction_small
credit-g


In [21]:
total_score.to_csv('total_score.csv')

In [13]:
total_place

,AUC place
Auto_ml,18
h2o,18
tpot,7
catboost-default,24
lightgbm-default,13
lightgbm-opt,25


In [14]:
res

,AUC
h2o,0.77646
lightgbm-default,0.77954
tpot,0.78164
catboost-default,0.78374
lightgbm-opt,0.78524
Auto_ml,0.79252


In [16]:
bench_result

{'Auto_ml': 18,
 'h2o': 18,
 'tpot': 7,
 'catboost-default': 24,
 'lightgbm-default': 13,
 'lightgbm-opt': 25}

In [17]:
bench = pd.io.json.json_normalize(bench_result,).T

In [18]:
bench.columns = ['AUC place']

In [19]:
bench = bench.sort_values(by='AUC place', ascending=False)

In [44]:
# import matplotlib.pyplot as plt
# plt.style.use('bmh')
# #import seaborn as sns
# #sns.set_style(style="darkgrid")

In [43]:
#plt.rcdefaults()
# fig, ax = plt.subplots()

# colors = color=['tab:blue', 'firebric',]

# ax.barh(bench.index, bench['AUC place'], align='center', color=colors,)
# ax.invert_yaxis()
# plt.show()

In [45]:
bench.to_csv('auc_place', index=False,)

,AUC place
lightgbm-opt,25
catboost-default,24
Auto_ml,18
h2o,18
lightgbm-default,13
tpot,7
